# Project: Semantic Search with Transformers

In this project, you’ll use the sentence_transformers library to perform semantic search on a corpus of machine learning papers. The 
sentence_transformers library enables us to easily generate embeddings for any text using Transformer-based models. Semantic similarity can then be 
modeled as the distance between two embeddings.

To complete this project, you’ll perform the following tasks:

1. Generate embeddings for each paper summary.
                      
2. Create an index for efficient search using Facebook’s Faiss library.
    
3. Test the search engine using custom prompts and summaries from the dataset.



## Task 1: Import the Libraries

Let’s start this project by importing the modules required for completing this project.

To complete this task, import the following libraries:

1. pandas: This module is used for loading and displaying the dataset.
                                                          
2. torch: This module is used to create and manipulate document embedding matrices.
                                                          
3. SentenceTransformer: This is a method in the sentence_transformers library and is used for retrieving the Transformer-based model.
                                                          
4. preprocessing: This is a submodule of the sklearn package and will be used to preprocess the data.

5. faiss: This library is used to create, store, and use the search index.

6. numpy: This package provides numerical computing capabilities and is useful for various calculations.

7. pickle: This module is used to load and store the model and embeddings.



In [2]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn import preprocessing
import faiss
import numpy as np
import pickle

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Task 2: Load the Data

First, let’s load the dataset as a pandas DataFrame. The dataset is made available as a file named arxivData.json in the ./usercode directory, 
and consists of the metadata of 41,000+ research papers.

Perform the following steps to complete this task:

1. Load the dataset to a pandas DataFrame.
    
2. Drop the author, link, and tag columns of the dataset.
    
3. Display the dataset header using the head() method.
    
4. Print the number of machine learning papers in the dataset.
    

In [3]:
pd.set_option('display.max_colwidth', None)
data = pd.read_json('/usercode/arxivData.json')
df = data.drop(columns=["author", "link", 'tag'])
print("Number of Machine Learning papers: ", df.id.unique().shape[0])
df.head()

Number of Machine Learning papers:  41000


,day,id,month,summary,title,year
0,1,1802.00209v1,2,"We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset.",Dual Recurrent Attention Units for Visual Question Answering,2018
1,12,1603.03827v1,3,"Recent approaches based on artificial neural networks (ANNs) have shown\npromising results for short-text classification. However, many short texts\noccur in sequences (e.g., sentences in a document or utterances in a dialog),\nand most existing ANN-based systems do not leverage the preceding short texts\nwhen classifying a subsequent one. In this work, we present a model based on\nrecurrent neural networks and convolutional neural networks that incorporates\nthe preceding short texts. Our model achieves state-of-the-art results on three\ndifferent datasets for dialog act prediction.",Sequential Short-Text Classification with Recurrent and Convolutional\n Neural Networks,2016
2,2,1606.00776v2,6,"We introduce the multiresolution recurrent neural network, which extends the\nsequence-to-sequence framework to model natural language generation as two\nparallel discrete stochastic processes: a sequence of high-level coarse tokens,\nand a sequence of natural language tokens. There are many ways to estimate or\nlearn the high-level coarse tokens, but we argue that a simple extraction\nprocedure is sufficient to capture a wealth of high-level discourse semantics.\nSuch procedure allows training the multiresolution recurrent neural network by\nmaximizing the exact joint log-likelihood over both sequences. In contrast to\nthe standard log- likelihood objective w.r.t. natural language tokens (word\nperplexity), optimizing the joint log-likelihood biases the model towards\nmodeling high-level abstractions. We apply the proposed model to the task of\ndialogue response generation in two challenging domains: the Ubuntu technical\nsupport domain, and Twitter conversations. On Ubuntu, the model outperforms\ncompeting approaches by a substantial margin, achieving state-of-the-art\nresults according to both automatic evaluation metrics and a human evaluation\nstudy. On Twitter, the model appears to generate more relevant and on-topic\nresponses according to automatic evaluation metrics. Finally, our experiments\ndemonstrate that the proposed model is more adept at overcoming the sparsity of\nnatural language and is better able to capture long-term structure.",Multiresolution Recurrent Neural Networks: An Application to Dialogue\n Response Generation,2016
3,23,1705.08142v2,5,"Multi-task learning is motivated by the observation that humans bring to bear\nwhat they know about related problems when solving new ones. Similarly, deep\nneural networks can profit from related tasks by sharing parameters with other\nnetworks. However, humans do not consciously decide to transfer knowledge\nbetween tasks. In Natural Language Processing (NLP), it is hard to predict if\nsharing will lead to improvements, particularly if tasks are only loosely\nrelated. To overcome this, we introduce Sluice Networks, a general framework\nfor multi-task learning where trainable parameters control the amount of\nsharing. Our framework generalizes previous proposals in enabling sharing of\nall combinations of subspaces, layers, and skip connections. We perform\nexperiments on three ta

In [4]:
data

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech Samek'}]",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.org/abs/1802.00209v1', 'type': 'text/html'}, {'rel': 'related', 'href': 'http://arxiv.org/pdf/1802.00209v1', 'type': 'application/pdf', 'title': 'pdf'}]",2,"We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset.","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.CL', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.CV', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.NE', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'stat.ML', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}]",Dual Recurrent Attention Units for Visual Question Answering,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck Dernoncourt'}]",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.org/abs/1603.03827v1', 'type': 'text/html'}, {'rel': 'related', 'href': 'http://arxiv.org/pdf/1603.03827v1', 'type': 'application/pdf', 'title': 'pdf'}]",3,"Recent approaches based on artificial neural networks (ANNs) have shown\npromising results for short-text classification. However, many short texts\noccur in sequences (e.g., sentences in a document or utterances in a dialog),\nand most existing ANN-based systems do not leverage the preceding short texts\nwhen classifying a subsequent one. In this work, we present a model based on\nrecurrent neural networks and convolutional neural networks that incorporates\nthe preceding short texts. Our model achieves state-of-the-art results on three\ndifferent datasets for dialog act prediction.","[{'term': 'cs.CL', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.AI', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.LG', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'cs.NE', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}, {'term': 'stat.ML', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}]",Sequential Short-Text Classification with Recurrent and Convolutional\n Neural Networks,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim Klinger'}, {'name': 'Gerald Tesauro'}, {'name': 'Kartik Talamadupula'}, {'name': 'Bowen Zhou'}, {'name': 'Yoshua Bengio'}, {'name': 'Aaron Courville'}]",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.org/abs/1606.00776v2', 'type': 'text/html'}, {'rel': 'related', 'href': 'http://arxiv.org/pdf/1606.00776v2', 'type': 'application/pdf', 'title': 'pdf'}]",6,"We introduce the multiresolution recurrent neural network, which extends the\nsequence-to-sequence framework to model natural language generation as two\nparallel discrete stochastic processes: a sequence of high-level coarse tokens,\nand a sequence of natural language tokens. There are many ways to estimate or\nlearn the high-level coarse tokens, but we argue that a simple extraction\nprocedure is sufficient to capture a wealth of high-level discourse semantics.\nSuch procedure allows training the multiresolution recurrent neural network by\nmaximizing the exact joint log-likelihood over both sequences. In contrast to\nthe standard log- likelihood objective w.r.t

## Task 3: Retrieve the Model

In this task, you’ll load the DistilBERT model using the sentence_transformers library. The sentence_transformers library allows us to use 
Transformer-based models from Huggingface, that are fine-tuned to generate semantically meaningful embedding matrices given natural language. 
The DistilBERT model is much smaller than the BERT model while having comparable performance and therefore are more suitable for our use case.

To complete this task, perform the following steps:

1. Use the SentenceTransformer() method from the sentence_transformers library to load a DistilBERT model. This method takes the name of the
Transformer-based model to load as a parameter.
                                                                                                                            
2. Move the model to the GPU if it is available. Print the device on which the model is located.
    

In [3]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))

print(model.device)

cpu


## Task 4: Generate or Load the Embeddings

In this task, you’ll either create or load the embeddings for all the paper summaries in the dataset.

To complete this task, perform the following steps:

1. Do one of the following:

To create the embeddings, use the encode() method to generate embedding vectors for the paper summaries using the DistilBERT model. Remember to 
save the embeddings using pickle.dump().

To load the embeddings that have been provided, open the default_embeddings.pickle file and load it to an embeddings object using the pickle.load()
method.

To load the embeddings that were generated in an earlier session, open the new_embeddings.pickle file and load it to an embeddings object using the
pickle.load() method.
    
2. Create the variable length, which is the number of embeddings in the index.

3. Print the shape of any one embedding.

N.B : The code given in this task for generating embeddings will create embeddings for the first 2000 records of the dataset.


In [4]:
embeddings = model.encode(df.summary.to_list()[:2000], show_progress_bar=True)
with open('/usercode/new_embeddings.pickle', 'wb') as pkl:
  pickle.dump(embeddings, pkl)

Batches: 100%|██████████| 63/63 [07:01<00:00,  6.68s/it]


In [5]:
with open('/usercode/default_embeddings.pickle', 'rb') as pkl:
  embeddings = pickle.load(pkl)

In [6]:
with open('/usercode/new_embeddings.pickle', 'rb') as pkl:
  embeddings = pickle.load(pkl)

In [7]:
length = len(embeddings)
length


2000

In [9]:
print('Shape of the one embedding: ', embeddings[0].shape)
print(embeddings.shape)

Shape of the one embedding:  (768,)
(2000, 768)


## Task 5: Data Preparation and Helper Methods

In this task, you’ll prepare the dataset by encoding the paper IDs as integers. You’ll then write a helper method for returning a list of the 
required dataset information, given a list of IDs.

To complete this task, perform the following steps:

1. The Faiss library requires integer IDs for the items present in the DataFrame. The dataset, on the other hand, has string IDs for each item.
Use the fit_transform() method of the sklearn.preprocessing.LabelEncoder class to convert the string item IDs to integer values.

2. Write a function named id2info(), which returns a list of column values for papers specified by their IDs. This method accepts the 
following parameters as input:

df: This is the DataFrame in which the data is contained.

I: This is a list of IDs of the papers for which the information is required.

column: This is the column of the DataFrame where the required information is stored.



In [10]:
le = preprocessing.LabelEncoder()
df['id'] = le.fit_transform(df['id'])
df.head(2)

,day,id,month,summary,title,year
0,1,36693,2,"We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset.",Dual Recurrent Attention Units for Visual Question Answering,2018
1,12,18198,3,"Recent approaches based on artificial neural networks (ANNs) have shown\npromising results for short-text classification. However, many short texts\noccur in sequences (e.g., sentences in a document or utterances in a dialog),\nand most existing ANN-based systems do not leverage the preceding short texts\nwhen classifying a subsequent one. In this work, we present a model based on\nrecurrent neural networks and convolutional neural networks that incorporates\nthe preceding short texts. Our model achieves state-of-the-art results on three\ndifferent datasets for dialog act prediction.",Sequential Short-Text Classification with Recurrent and Convolutional\n Neural Networks,2016


In [11]:
def id2info(df, I, column):
    return [list(df[df.id == idx][column]) for idx in I]

## Task 6: Set up the Index

In this task, you will set up the search index using the Faiss library.

To complete this task, perform the following steps:

1. Convert the embeddings to NumPy arrays of the float32 data type.

2. Initialize the index using the IndexFlatL2() method from the Faiss library. This method will take the length of the embeddings as input. This 
index will return search results based on the k-nearest-neighbors algorithm through a brute-force search with L2 (Euclidean) distances.
    
3. Use the IndexIDMap() from the Faiss library to create an index map that encapsulates the initialized index and provides a mapping between IDs and 
the embedding vectors when adding and searching. This method will take the index that was created in the previous step as a parameter.

4. Use the add_with_ids() method to add the embeddings and their IDs to the index map. This method will take the following parameters:

embeddings: An array of embedding vectors

xids: The IDs corresponding to the embedding vectors

5. Print the number of embeddings in the index map.



In [12]:
embeddings = np.array(embeddings).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, df['id'][:length])

print("Number of embeddings in the Faiss index: ", index.ntotal)

Number of embeddings in the Faiss index:  2000


## Task 7: Search with a Summary

In this section, you’ll search the index with a summary from the dataset. To complete this task, perform the following steps:

1. Print the summary that will be used to search.

2. Get the 10 nearest neighbors by searching the index map. Search the index by using the index.search() method.

This method accepts the following arguments:

vector: These are the embeddings of the summary that will be used to search.

k: This is the number of neighbors the model will return.
    
The method will return the following outputs:

D: A list of L2 (Euclidean) distances of the results.

I: The IDs of the results.
    
3. Display the distances, IDs, titles, and summaries of the returned results as a DataFrame.


In [13]:
df.iloc[1337, [3, 1]]

summary    In this paper we study the application of convolutional neural networks for\njointly detecting objects depicted in still images and estimating their 3D\npose. We identify different feature representations of oriented objects, and\nenergies that lead a network to learn this representations. The choice of the\nrepresentation is crucial since the pose of an object has a natural, continuous\nstructure while its category is a discrete variable. We evaluate the different\napproaches on the joint object detection and pose estimation task of the\nPascal3D+ benchmark using Average Viewpoint Precision. We show that a\nclassification approach on discretized viewpoints achieves state-of-the-art\nperformance for joint object detection and pose estimation, and significantly\noutperforms existing baselines on this benchmark.
id                                                                                                                                                                     

In [14]:
df.iloc[0, [3, 1]]

summary    We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset.
id                                                                                                                                                                                                                        

In [15]:
D, I = index.search(np.array([embeddings[1337]]), k=10)
pd.DataFrame({'L2 distance': D.flatten().tolist(), 'ML paper IDs': I.flatten().tolist(), 'ML paper titles': id2info(df, I.flatten(), 'title'), 'Summaries': id2info(df, I.flatten(), 'summary')}).head(10)

,L2 distance,ML paper IDs,ML paper titles,Summaries
0,0.000000,12964,[Convolutional Neural Networks for joint object detection and pose\n estimation: A comparative study],"[In this paper we study the application of convolutional neural networks for\njointly detecting objects depicted in still images and estimating their 3D\npose. We identify different feature representations of oriented objects, and\nenergies that lead a network to learn this representations. The choice of the\nrepresentation is crucial since the pose of an object has a natural, continuous\nstructure while its category is a discrete variable. We evaluate the different\napproaches on the joint object detection and pose estimation task of the\nPascal3D+ benchmark using Average Viewpoint Precision. We show that a\nclassification approach on discretized viewpoints achieves state-of-the-art\nperformance for joint object detection and pose estimation, and significantly\noutperforms existing baselines on this benchmark.]"
1,61.530529,11503,[Deep Metric Learning for Practical Person Re-Identification],"[Various hand-crafted features and metric learning methods prevail in the\nfield of person re-identification. Compared to these methods, this paper\nproposes a more general way that can learn a similarity metric from image\npixels directly. By using a ""siamese"" deep neural network, the proposed method\ncan jointly learn the color feature, texture feature and metric in a unified\nframework. The network has a symmetry structure with two sub-networks which are\nconnected by Cosine function. To deal with the big variations of person images,\nbinomial deviance is used to evaluate the cost between similarities and labels,\nwhich is proved to be robust to outliers.\n Compared to existing researches, a more practical setting is studied in the\nexperiments that is training and test on different datasets (cross dataset\nperson re-identification). Both in ""intra dataset"" and ""cross dataset""\nsettings, the superiorities of the proposed method are illustrated on VIPeR and\nPRID.]"
2,65.805099,11377,[Cortical spatio-temporal dimensionality reduction for visual grouping],"[The visual systems of many mammals, including humans, is able to integrate\nthe geometric information of visual stimuli and to perform cognitive tasks\nalready at the first stages of the cortical processing. This is thought to be\nthe result of a combination of mechanisms, which include feature extraction at\nsingle cell level and geometric processing by means of cells connectivity. We\npresent a geometric model of such connectivities in the space of detected\nfeatures associated to spatio-temporal visual stimuli, and show how they can be\nused to obtain low-level object segmentation. The main idea is that of defining\na spectral clustering procedure with anisotropic affinities over datasets\nconsisting of embeddings of the visual stimuli into higher dimensional spaces.\nNeural plausibility of the proposed arguments will be discussed.]"
3,67.032898,11142,[Heterogeneous Multi-task Learning for Human Pose Estimation with Deep\n Convolutional Neural Network],"[We propose an heterogeneous multi-task learning framework for human pose\nestimation from monocular image with deep convolutional neural network. In\nparticular, we simultaneously learn a pose-joint regressor and a sliding-window\nbody-part detector in a deep network architecture. We show that including the\nbody-part detection task helps to regularize the network, directing it to\nconverge to a good solution. We report competitive and state-of-art results on\nseveral data sets. We also empirically show that the learned neurons in the\nmiddle layer of our network are tuned to localized body parts.]"
4,69.937653,30623,[Neural Expectation Maximization],"[Many real world tasks such as reasoning and physical interaction require\nidentification and manipulation of conceptual entities. A first step towards\nsolving these tasks is the automated discovery of distri

## Task 8: Search with a Prompt

In this task, you’ll search the dataset using a prompt. To complete this task, perform the following steps:

1. Create the prompt and assign it to a string variable named user_query.
    
2. Encode the prompt using the encode() method of the selected. This method accepts the prompt as a list.
    
3. Search the index using the generated embeddings, and display the distances, IDs, titles, and summaries in the results as a DataFrame.


In [16]:

user_query = "The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data"



In [17]:
embed = model.encode(list(user_query))
D, I = index.search(np.array([embed]).squeeze().astype("float32"), k=10)

results = {'L2 distances':D.flatten().tolist(), 'ML paper IDs':I.flatten().tolist(), "Titles": id2info(df, I.flatten(), 'title'), "Summaries": id2info(df, I.flatten(), 'summary')}

pd.DataFrame(results).head(10)

,L2 distances,ML paper IDs,Titles,Summaries
0,398.835114,26890,[Abstract Syntax Networks for Code Generation and Semantic Parsing],"[Tasks like code generation and semantic parsing require mapping unstructured\n(or partially structured) inputs to well-formed, executable outputs. We\nintroduce abstract syntax networks, a modeling framework for these problems.\nThe outputs are represented as abstract syntax trees (ASTs) and constructed by\na decoder with a dynamically-determined modular structure paralleling the\nstructure of the output tree. On the benchmark Hearthstone dataset for code\ngeneration, our model obtains 79.2 BLEU and 22.7% exact match accuracy,\ncompared to previous state-of-the-art values of 67.1 and 6.1%. Furthermore, we\nperform competitively on the Atis, Jobs, and Geo semantic parsing datasets with\nno task-specific engineering.]"
1,401.845093,30018,[Dual Rectified Linear Units (DReLUs): A Replacement for Tanh Activation\n Functions in Quasi-Recurrent Neural Networks],"[In this paper, we introduce a novel type of Rectified Linear Unit (ReLU),\ncalled a Dual Rectified Linear Unit (DReLU). A DReLU, which comes with an\nunbounded positive and negative image, can be used as a drop-in replacement for\na tanh activation function in the recurrent step of Quasi-Recurrent Neural\nNetworks (QRNNs) (Bradbury et al. (2017)). Similar to ReLUs, DReLUs are less\nprone to the vanishing gradient problem, they are noise robust, and they induce\nsparse activations.\n We independently reproduce the QRNN experiments of Bradbury et al. (2017) and\ncompare our DReLU-based QRNNs with the original tanh-based QRNNs and Long\nShort-Term Memory networks (LSTMs) on sentiment classification and word-level\nlanguage modeling. Additionally, we evaluate on character-level language\nmodeling, showing that we are able to stack up to eight QRNN layers with\nDReLUs, thus making it possible to improve the current state-of-the-art in\ncharacter-level language modeling over shallow architectures based on LSTMs.]"
2,403.243622,7184,[KSU KDD: Word Sense Induction by Clustering in Topic Space],"[We describe our language-independent unsupervised word sense induction\nsystem. This system only uses topic features to cluster different word senses\nin their global context topic space. Using unlabeled data, this system trains a\nlatent Dirichlet allocation (LDA) topic model then uses it to infer the topics\ndistribution of the test instances. By clustering these topics distributions in\ntheir topic space we cluster them into different senses. Our hypothesis is that\ncloseness in topic space reflects similarity between different word senses.\nThis system participated in SemEval-2 word sense induction and disambiguation\ntask and achieved the second highest V-measure score among all other systems.]"
3,403.410858,28659,[Topic supervised non-negative matrix factorization],"[Topic models have been extensively used to organize and interpret the\ncontents of large, unstructured corpora of text documents. Although topic\nmodels often perform well on traditional training vs. test set evaluations, it\nis often the case that the results of a topic model do not align with human\ninterpretation. This interpretability fallacy is largely due to the\nunsupervised nature of topic models, which prohibits any user guidance on the\nresults of a model. In this paper, we introduce a semi-supervised method called\ntopic supervised non-negative matrix factorization (TS-NMF) that enables the\nuser to provide labeled example documents to promote the discovery of more\nmeaningful semantic structure of a corpus. In this way, the results of TS-NMF\nbetter match the intuition and desired labeling of the user. The core of TS-NMF\nrelies on solving a non-convex optimization problem for which we derive an\niterative algorithm that is shown to be monotonic and convergent to a local\noptimum. We demonstrate the practical utility of TS-NMF on the Reuters and\nPubMed corpora, and find that TS-N

# End